In [30]:
using JuMP
using GLPK

- I am 3 digit number between 400 and 800
- My digits add up to 15
- My ten's digit is twice hundereds' digit
- What number am I?

$$
\begin{aligned}
\min z = & d_1^- + d_1^+ \\
\text{Subject to:} \\
& x_1 + x_2 + x_3 + d_1^- - d_1^+ = 15 \\
& x_2 - 2x_1 = 0 \\
& 100x_1 + 10x_2 + x_3 \le 800 \\
& 100x_1 + 10x_2 + x_3 \ge 400 \\
& x_1, x_2, x_3 \ge 0 \\
& x_1, x_2, x_3 \le 9 \\
& d_1^-, d_1^+ \ge 0 \\
& x_1, x_2, x_3 \in \{0, Z^+\}
\end{aligned}
$$

In [31]:
using JuMP, GLPK

m = Model(GLPK.Optimizer)

@variable(m, x[1:3], Int)
@variable(m, d[1:2])

@objective(m, Min, sum(d[1:2]))
@constraint(m, x[1] + x[2] + x[3] + d[1] - d[2] == 15)
@constraint(m, x[2] == 2 * x[1])
@constraint(m, x[1] * 100 + x[2] *10 + x[3] <= 800)
@constraint(m, x[1] * 100 + x[2] *10 + x[3] >= 400)
@constraint(m, x[1:3] .>= 0)
@constraint(m, x[1:3] .<= 9)
@constraint(m, d[1:2] .>= 0)

optimize!(m)

In [32]:
value.(x[1:3])

3-element Vector{Float64}:
 4.0
 8.0
 3.0